In [2]:
!git clone https://github.com/xinntao/ESRGAN

Cloning into 'ESRGAN'...
remote: Enumerating objects: 225, done.
remote: Total 225 (delta 0), reused 0 (delta 0), pack-reused 225
Receiving objects: 100% (225/225), 24.86 MiB | 9.14 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [ ]:
# USE DATA FROM COPERNICUS_DATA, COMPARE TO HR

In [3]:
import torch
import fnmatch
import os
from os import listdir,makedirs
from os.path import isfile,join

print(torch.version.cuda)
print(torch.__version__)
print(torch.cuda.get_device_name(device=None))

12.1
2.2.1+cu121
NVIDIA A100-SXM4-40GB


In [ ]:
# !cp -r '/content/drive/MyDrive/csc_496FinalProject/ESRGAN/models/RRDB_ESRGAN_x4.pth' '/content/ESRGAN/models'

In [ ]:
# LR_Data = '/content/drive/MyDrive/csc_496FinalProject/Data/Copernicus_data/remote_sensing/train/LR'

In [ ]:
# move this to repo
!find "/content/drive/MyDrive/csc_496FinalProject/Data/Copernicus_data/remote_sensing/train/LR/" -type f -name '*.jpg' -exec cp {} "/content/ESRGAN/LR/" \;

In [ ]:
!python /content/ESRGAN/test.py

Traceback (most recent call last):
  File "/content/ESRGAN/test.py", line 15, in <module>
    model.load_state_dict(torch.load(model_path), strict=True)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 998, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 445, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 426, in __init__
    super().__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: 'models/RRDB_ESRGAN_x4.pth'


In [4]:
%cd ESRGAN

/content/ESRGAN


In [ ]:
import os.path as osp
import glob
import cv2
import numpy as np
import torch
import RRDBNet_arch as arch

model_path = '/content/drive/MyDrive/csc_496FinalProject/ESRGAN/models/RRDB_ESRGAN_x4.pth'
device = torch.device('cuda')  # if you want to run on CPU, change 'cuda' -> cpu
# device = torch.device('cpu')
test_img_folder = '/content/drive/MyDrive/csc_496FinalProject/Data/Copernicus_data/remote_sensing/train/LR/*'

model = arch.RRDBNet(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

print('Model path {:s}. \nTesting...'.format(model_path))

idx = 0
for path in glob.glob(test_img_folder):
    idx += 1
    base = osp.splitext(osp.basename(path))[0]
    print(f"[{idx}] Processing image: {base}")

    # read images
    print(f"Reading image: {path}")
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)
    print(f"Image shape: {img_LR.shape}")

    with torch.no_grad():
        print("Running model inference...")
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    print(f"Saving result to: /content/drive/MyDrive/csc_496FinalProject/ESRGAN/results/{base}_rlt.png")
    cv2.imwrite(f'/content/drive/MyDrive/csc_496FinalProject/ESRGAN/results/{base}_rlt.png', output)

Model path /content/drive/MyDrive/csc_496FinalProject/ESRGAN/models/RRDB_ESRGAN_x4.pth. 
Testing...
[1] Processing image: S2A_MSIL2A_20240503T084601_N0510_R107_T33KVT_20240503T140450-ql
Reading image: /content/drive/MyDrive/csc_496FinalProject/Data/Copernicus_data/remote_sensing/train/LR/S2A_MSIL2A_20240503T084601_N0510_R107_T33KVT_20240503T140450-ql.jpg
Image shape: torch.Size([1, 3, 256, 256])
Running model inference...
Saving result to: /content/drive/MyDrive/csc_496FinalProject/ESRGAN/results/S2A_MSIL2A_20240503T084601_N0510_R107_T33KVT_20240503T140450-ql_rlt.png
[2] Processing image: S2B_MSIL2A_20240502T082559_N0510_R021_T34KDD_20240502T113605-ql
Reading image: /content/drive/MyDrive/csc_496FinalProject/Data/Copernicus_data/remote_sensing/train/LR/S2B_MSIL2A_20240502T082559_N0510_R021_T34KDD_20240502T113605-ql.jpg
Image shape: torch.Size([1, 3, 256, 256])
Running model inference...
Saving result to: /content/drive/MyDrive/csc_496FinalProject/ESRGAN/results/S2B_MSIL2A_20240502T0825

Compare to HR images. Compute metrics

In [ ]:
import matplotlib.pyplot as plt

lr_folder = '/content/drive/MyDrive/csc_496FinalProject/Data/Copernicus_data/remote_sensing/train/LR/'
hr_folder = '/content/drive/MyDrive/csc_496FinalProject/Data/Copernicus_data/remote_sensing/train/HR/'
gen_folder = '/content/drive/MyDrive/csc_496FinalProject/ESRGAN/results/'

# Get the list of image names in the LR folder
lr_img_names = os.listdir(lr_folder)
count = 0
# Iterate over the LR images
for img_name in lr_img_names:
    count+=1
    lr_path = os.path.join(lr_folder, img_name)
    img_LR = cv2.imread(lr_path, cv2.IMREAD_COLOR)

    hr_path = os.path.join(hr_folder, img_name)
    img_HR = cv2.imread(hr_path, cv2.IMREAD_COLOR)

    gen_path = os.path.join(gen_folder, img_name[:-4] + '_rlt.png')
    img_gen = cv2.imread(gen_path, cv2.IMREAD_COLOR)

    fig, axs = plt.subplots(1, 3, figsize=(20, 20))
    axs[0].imshow(cv2.cvtColor(img_LR, cv2.COLOR_BGR2RGB))
    axs[0].set_title('Low Resolution')
    axs[1].imshow(cv2.cvtColor(img_gen, cv2.COLOR_BGR2RGB))
    axs[1].set_title('Generated HR')
    axs[2].imshow(cv2.cvtColor(img_HR, cv2.COLOR_BGR2RGB))
    axs[2].set_title('High Resolution')
    plt.tight_layout()
    plt.savefig(os.path.join(gen_folder, f'ESRGAN_LRvsSRvsHRPlot_{count}.png'))
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

compute metrics: Peak Signal-to-Noise Ratio (PSNR) and Structural Similarity Index (SSIM)

In [ ]:
import os
import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

hr_folder = '/content/drive/MyDrive/csc_496FinalProject/Data/Copernicus_data/remote_sensing/train/HR/'
gen_folder = '/content/drive/MyDrive/csc_496FinalProject/ESRGAN/results/'

hr_img_names = os.listdir(hr_folder)

psnr_values = []
ssim_values = []


for img_name in hr_img_names:
    hr_path = os.path.join(hr_folder, img_name)
    img_HR = cv2.imread(hr_path, cv2.IMREAD_COLOR)

    gen_path = os.path.join(gen_folder, img_name[:-4] + '_rlt.png')
    img_gen = cv2.imread(gen_path, cv2.IMREAD_COLOR)
    # resize generated image to High-res size
    img_gen_downscaled = cv2.resize(img_gen, (img_HR.shape[1], img_HR.shape[0]), interpolation=cv2.INTER_CUBIC)

    psnr = peak_signal_noise_ratio(img_HR, img_gen_downscaled)
    psnr_values.append(psnr)

    ssim = structural_similarity(img_HR, img_gen_downscaled, channel_axis=2)
    ssim_values.append(ssim)

# find averages
avg_psnr = np.mean(psnr_values)
avg_ssim = np.mean(ssim_values)

print(f"Average PSNR: {avg_psnr:.2f} dB")
print(f"Average SSIM: {avg_ssim:.4f}")

Average PSNR: 32.09 dB
Average SSIM: 0.8563


Re-do to make comparable to results from ESRGAN_finetune.ipynb

In [29]:
%mkdir /content/drive/MyDrive/csc_496FinalProject/ESRGAN/results_tif

In [ ]:
# Prep data. Change LR test data from 128x128 .tif file to 256x256 .jpg file
# tried without doing this and leads to weird results
# stored new in path: /content/drive/MyDrive/csc_496FinalProject/Data/Copernicus_data/Copernicus_jpg_ims/LR

In [30]:
import os.path as osp
import glob
import cv2
import numpy as np
import torch
import RRDBNet_arch as arch

model_path = '/content/drive/MyDrive/csc_496FinalProject/ESRGAN/models/RRDB_ESRGAN_x4.pth'
device = torch.device('cuda')  # if you want to run on CPU, change 'cuda' -> cpu
# device = torch.device('cpu')
test_img_folder = '/content/drive/MyDrive/csc_496FinalProject/Data/Copernicus_data/remote_sensing/test/LR/*'

model = arch.RRDBNet(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

print('Model path {:s}. \nTesting...'.format(model_path))

idx = 0
for path in glob.glob(test_img_folder):
    idx += 1
    base = osp.splitext(osp.basename(path))[0]
    print(f"[{idx}] Processing image: {base}")

    # read images
    print(f"Reading image: {path}")
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)
    print(f"Image shape: {img_LR.shape}")

    with torch.no_grad():
        print("Running model inference...")
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    print(f"Saving result to: /content/drive/MyDrive/csc_496FinalProject/ESRGAN/results_new/{base}_rlt.png")
    cv2.imwrite(f'/content/drive/MyDrive/csc_496FinalProject/ESRGAN/results_tif/{base}_rlt.png', output)

Model path /content/drive/MyDrive/csc_496FinalProject/ESRGAN/models/RRDB_ESRGAN_x4.pth. 
Testing...
[1] Processing image: LR2_test
Reading image: /content/drive/MyDrive/csc_496FinalProject/Data/Copernicus_data/remote_sensing/test/LR/LR2_test.tif
Image shape: torch.Size([1, 3, 128, 128])
Running model inference...
Saving result to: /content/drive/MyDrive/csc_496FinalProject/ESRGAN/results_new/LR2_test_rlt.png
[2] Processing image: LR3_test
Reading image: /content/drive/MyDrive/csc_496FinalProject/Data/Copernicus_data/remote_sensing/test/LR/LR3_test.tif
Image shape: torch.Size([1, 3, 128, 128])
Running model inference...
Saving result to: /content/drive/MyDrive/csc_496FinalProject/ESRGAN/results_new/LR3_test_rlt.png
[3] Processing image: LR1_test
Reading image: /content/drive/MyDrive/csc_496FinalProject/Data/Copernicus_data/remote_sensing/test/LR/LR1_test.tif
Image shape: torch.Size([1, 3, 128, 128])
Running model inference...
Saving result to: /content/drive/MyDrive/csc_496FinalProject/

In [33]:
import matplotlib.pyplot as plt
import cv2
import os
from PIL import Image

# Paths
hr_folder = '/content/drive/MyDrive/csc_496FinalProject/Data/Copernicus_data/remote_sensing/test/HR/'
lr_folder = '/content/drive/MyDrive/csc_496FinalProject/Data/Copernicus_data/remote_sensing/test/LR/'
gen_folder = '/content/drive/MyDrive/csc_496FinalProject/ESRGAN/results_tif/'

# Lists of images
hr_img_names = sorted(os.listdir(hr_folder))
lr_img_names = sorted(os.listdir(lr_folder))
gen_img_names = sorted(os.listdir(gen_folder))

# Set up the plot
fig, axs = plt.subplots(nrows=len(hr_img_names), ncols=3, figsize=(15, 5 * len(hr_img_names)))  # Adjusted for better visibility

# Iterate over each high-resolution image
for i, hr_img_name in enumerate(hr_img_names):
    base_name = hr_img_name.split('.')[0].replace('HR', 'LR')
    lr_img_name = f"{base_name}.tif"  # Assuming LR images have the same base name but different prefix and extension
    gen_img_name = f"{base_name}_rlt.png"

    # Load HR image
    hr_path = os.path.join(hr_folder, hr_img_name)
    img_HR = Image.open(hr_path)
    img_HR = np.array(img_HR)

    # Load LR image
    lr_path = os.path.join(lr_folder, lr_img_name)
    img_LR = Image.open(lr_path)
    img_LR = np.array(img_LR)

    # Load Generated image
    gen_path = os.path.join(gen_folder, gen_img_name)
    img_gen = cv2.imread(gen_path, cv2.IMREAD_COLOR)
    if img_gen is None:
        print(f"Failed to load generated image: {gen_path}")

    # Display the images
    axs[i, 0].imshow(cv2.cvtColor(img_LR, cv2.COLOR_BGR2RGB))
    axs[i, 0].set_title(f"Low-Res: {lr_img_name}")

    axs[i, 1].imshow(cv2.cvtColor(img_gen, cv2.COLOR_BGR2RGB))
    axs[i, 1].set_title(f"Generated: {gen_img_name}")

    axs[i, 2].imshow(img_HR)
    axs[i, 2].set_title(f"High-Res: {hr_img_name}")

plt.tight_layout()
plt.show()


Check Image size

EXPECTING

          LR: 256x256
          
          HR: 512x512
          
          GEN:1024x1024


In [7]:
import os
from PIL import Image

def check_image_sizes(dataset_path):
    train_lr_folder = os.path.join(dataset_path, "train", "LR")
    train_hr_folder = os.path.join(dataset_path, "train", "HR")
    print("Training images:")
    check_folder_image_sizes(train_lr_folder, "Low-resolution")
    check_folder_image_sizes(train_hr_folder, "High-resolution")

    test_lr_folder = os.path.join(dataset_path, "test", "LR")
    test_hr_folder = os.path.join(dataset_path, "test", "HR")
    print("Test images:")
    check_folder_image_sizes(test_lr_folder, "Low-resolution")
    check_folder_image_sizes(test_hr_folder, "High-resolution")

def check_folder_image_sizes(folder_path, resolution):
    print(f"{resolution} images:")
    image_sizes = set()
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        with Image.open(file_path) as image:
            width, height = image.size
            image_sizes.add((width, height))
    print(f"Unique image sizes: {image_sizes}")
    print()

# Specify the path to your dataset
your_dataset_path = "/content/drive/MyDrive/csc_496FinalProject/Data/Copernicus_data/remote_sensing"

# Check image sizes in your dataset
check_image_sizes(your_dataset_path)

Training images:
Low-resolution images:
Unique image sizes: {(128, 128)}

High-resolution images:
Unique image sizes: {(512, 512)}

Test images:
Low-resolution images:
Unique image sizes: {(128, 128)}

High-resolution images:
Unique image sizes: {(512, 512)}



In [25]:
import os
import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from PIL import Image
from skimage import img_as_float32

# Directories
hr_folder = '/content/drive/MyDrive/csc_496FinalProject/Data/Copernicus_data/remote_sensing/test/HR/'
gen_folder = '/content/drive/MyDrive/csc_496FinalProject/ESRGAN/results_new/'

# List HR and Generated images
hr_img_names = os.listdir(hr_folder)
gen_img_names = os.listdir(gen_folder)

# Dictionaries to hold PSNR and SSIM values
psnr_values = []
ssim_values = []

# Process each HR image
for hr_img_name in hr_img_names:
    # Construct HR image path
    hr_path = os.path.join(hr_folder, hr_img_name)
    img_HR = Image.open(hr_path)
    img_HR = np.array(img_HR)
    if img_HR.ndim == 2:  # Convert grayscale to RGB if necessary
        img_HR = cv2.cvtColor(img_HR, cv2.COLOR_GRAY2RGB)
    img_HR = img_as_float32(img_HR)  # Convert image to float32 for skimage functions

    # Extract base name for matching
    base_name = hr_img_name.split('.')[0].replace('HR', 'LR')  # Assuming naming convention HRX_test -> LRX_test

    # Find corresponding generated image
    gen_img_name = [name for name in gen_img_names if name.startswith(base_name) and name.endswith('_rlt.png')]
    if not gen_img_name:
        print(f"No generated image found for {hr_img_name}")
        continue
    gen_img_name = gen_img_name[0]
    gen_path = os.path.join(gen_folder, gen_img_name)
    img_gen = cv2.imread(gen_path, cv2.IMREAD_COLOR)
    if img_gen is None:
        print(f"Failed to load generated image: {gen_path}")
        continue

    # Compute PSNR and SSIM
    psnr = peak_signal_noise_ratio(img_HR, img_gen)
    ssim = structural_similarity(img_HR, img_gen, multichannel=True)

    # Store results
    psnr_values.append(psnr)
    ssim_values.append(ssim)

# Calculate averages
avg_psnr = np.mean(psnr_values)
avg_ssim = np.mean(ssim_values)

print(f"Average PSNR: {avg_psnr:.2f} dB")
print(f"Average SSIM: {avg_ssim:.4f}")


<ipython-input-25-cc96b8b75a42>:46: UserWarning: Inputs have mismatched dtype.  Setting data_range based on image_true.
  psnr = peak_signal_noise_ratio(img_HR, img_gen)
<ipython-input-25-cc96b8b75a42>:47: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim = structural_similarity(img_HR, img_gen, multichannel=True)
/usr/local/lib/python3.10/dist-packages/skimage/_shared/utils.py:348: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
  return func(*args, **kwargs)


Average PSNR: -41.98 dB
Average SSIM: 0.0001
